In [1]:
import pandas as pd
import numpy as np
import pickle
import time

import random
import itertools
from scipy import spatial

In [2]:
%%time
emb_df = pd.read_csv('data/embedding_large_128.emb', delimiter=' ', skiprows=1, header=None).set_index(0)
emb_dict = dict(zip(emb_df.index.to_numpy(), emb_df.to_numpy()))
del(emb_df)
print(len(emb_dict))
emb_dict[5]

6280772
CPU times: user 1min 19s, sys: 4.99 s, total: 1min 24s
Wall time: 1min 24s


array([ 0.839156 ,  2.00218  ,  0.0821642, -0.308378 , -3.15148  ,
        3.02667  ,  2.29951  ,  2.44862  ,  2.25199  , -0.467363 ,
        1.85577  ,  1.67371  , -3.06084  ,  2.73344  , -5.25429  ,
        3.79478  ,  1.32492  , -3.59908  , -4.11551  ,  2.59807  ,
        0.924388 ,  1.62767  ,  8.06372  ,  1.98649  , -3.39114  ,
       -2.0659   ,  3.49028  , -2.65928  ,  0.0121457, -1.53298  ,
       -7.14562  ,  5.93844  , -0.389844 ,  0.0615805, -5.3076   ,
        0.727857 , -1.32486  ,  0.995462 ,  5.25318  ,  1.71723  ,
        2.22789  ,  2.49593  ,  3.30571  , -1.80363  ,  2.56975  ,
        2.61248  , -2.86709  , -1.25002  ,  1.7451   , -0.357759 ,
        1.86635  ,  2.68591  ,  7.39115  ,  2.15686  ,  0.820224 ,
        1.22685  ,  0.584724 ,  1.17428  , -2.25298  ,  2.14346  ,
        7.07783  , -0.114723 , -6.72226  ,  3.55058  ,  2.05364  ,
        0.588275 ,  3.27242  , -1.54624  ,  1.39369  ,  3.44337  ,
       -2.46174  ,  4.08027  ,  1.95556  ,  1.47609  , -5.4742

In [3]:
%%time
dataset = pickle.load(open("data/dataset.p", "rb"))
print(len(dataset))

def get_samples(dataset, n, seed=None):
    random.seed(seed)
    return random.sample(dataset, n)

get_samples(dataset, 1, 1)

240000
CPU times: user 4.37 s, sys: 248 ms, total: 4.62 s
Wall time: 4.62 s


[([array([     198,      361,      362,      363,   103495,   154573,
            181902,   208761,   215669,   310802,   464176,   553157,
            606950,   671914,   672123,   695497,   704886,   705571,
            789406,   841559,   849205,   910261,   973422,  1063884,
           1286106,  1378456,  1466634,  1765465,  2054700,  2064979,
           2080106,  2413988,  2519948,  2587247,  2605509,  2631661,
           2667316,  2918355,  3136905,  3137058,  3657549,  3884054,
           4241182,  4322106,  4633649,  4745451,  4969484,  5865994,
           6734846,  7258664,  7892527,  7928600, 12403849, 13409491,
          16385874, 23003077, 23041506, 30324643]),
   [None],
   array([      43,    12560,    36730,    74740,    84072,   120304,
            193383,   212940,   224606,   268639,   330961,   339805,
            372199,   435897,   435966,   491507,   527449,   539183,
            556079,   559336,   560816,   602652,   654493,   698985,
            729125,   80988

In [4]:
id_pop_df = pd.read_csv("data/id_counts.csv").set_index('item_id')
display(id_pop_df)
item_pop_dict = id_pop_df.views.to_dict()
del(id_pop_df)
print(item_pop_dict[6199])

,page_id,title,views,counts
item_id,,,,
6199,12,Anarchism,31335,3540
38404,25,Autism,49693,2114
101038,39,Albedo,14573,2825
9659,290,A,25859,175
173,303,Alabama,52765,11125
...,...,...,...,...
76894635,62470350,Daming Zhu,16,0
76894633,62470423,Tony Dews,7,2
76896959,62470432,Samsung PL20,9,0


31335


In [5]:
def get_cos(item1, item2):
    try:
        return spatial.distance.cosine(emb_dict[item1], emb_dict[item2])
    except KeyError:
        return 1

def sort_ids(candidate_ids, n):
    if len(candidate_ids) == 0:
        return candidate_ids
    tie_break_list = [item_pop_dict.get(i, 0) for i in candidate_ids]
    
    return candidate_ids[np.argsort(tie_break_list)[::-1]][:n]

# 9036 Nikola Tesla
# 317521 Elon Musk
# 478214 Tesla Car Company
print(get_cos(9036, 478214))
print(get_cos(317521, 478214))

0.0003447095869324457
0.0009393673295353677


In [6]:
%%time
def reduce_neighbors(sample, max_neighbors, max_neighbor_ids, mode='closest'):
    temp = [i for i in sample if len(i)>0]
    
    if len(temp) <= max_neighbors+1:
        temp = [i for i in temp if i[0] is not None]
    else:
        if mode == 'closest':
            for i, j in enumerate(temp):
                if j[0] is None:
                    if i == 0 or i == len(temp):
                        temp = [i for i in temp if i[0] is not None][:max_neighbors]
                    else:
                        left = temp[:i]
                        right = temp[i+1:]
                        if len(left) <= max_neighbors/2:
                            temp = left + right[:max_neighbors-len(left)]
                        elif len(right) <= max_neighbors/2:
                            temp = left[-(max_neighbors-len(right)):] + right
                        else:
                            left = left[-max_neighbors//2:]
                            temp = left + right[:max_neighbors-len(left)]
                    break
        if mode == 'random' or len(temp) > max_neighbors:
            temp = [i for i in temp if i[0] is not None]
            temp = [temp[i] for i in np.random.randint(len(temp), size=max_neighbors)]
            
    temp = [sort_ids(i, max_neighbor_ids) for i in temp]
    return temp

sample = get_samples(dataset, 1, 1)
display(sample)
reduced_sample = reduce_neighbors(sample[0], 2, 8)
reduced_sample

[([array([     198,      361,      362,      363,   103495,   154573,
            181902,   208761,   215669,   310802,   464176,   553157,
            606950,   671914,   672123,   695497,   704886,   705571,
            789406,   841559,   849205,   910261,   973422,  1063884,
           1286106,  1378456,  1466634,  1765465,  2054700,  2064979,
           2080106,  2413988,  2519948,  2587247,  2605509,  2631661,
           2667316,  2918355,  3136905,  3137058,  3657549,  3884054,
           4241182,  4322106,  4633649,  4745451,  4969484,  5865994,
           6734846,  7258664,  7892527,  7928600, 12403849, 13409491,
          16385874, 23003077, 23041506, 30324643]),
   [None],
   array([      43,    12560,    36730,    74740,    84072,   120304,
            193383,   212940,   224606,   268639,   330961,   339805,
            372199,   435897,   435966,   491507,   527449,   539183,
            556079,   559336,   560816,   602652,   654493,   698985,
            729125,   80988

TypeError: unhashable type: 'numpy.ndarray'

In [62]:
def disambiguate(sample, max_neighbors, max_neighbor_ids, max_root_ids, \
                 skip_nonroot=False, tune=0, baseline=False, verbose=False):
    if verbose: print(sample)
    st = time.time()
    
    true_id = sample[2][2]
    candidate_ids = sort_ids(sample[0][0], max_root_ids)
    neighbor_ids = [i for i in sample[0][1:] if len(i)>0]
        
    if len(candidate_ids) <= 1 or len(neighbor_ids) <= 1 or baseline:
        return candidate_ids[0] == true_id
    
    neighbor_ids = reduce_neighbors(neighbor_ids, max_neighbors, max_neighbor_ids)
    
    comb_ids = [candidate_ids]+neighbor_ids
    combination_list = list(itertools.product(*comb_ids))
    #print(combination_list)
    if verbose: print(len(combination_list), len(list(itertools.combinations(combination_list[0], 2))))
    
    if len(combination_list) * len(list(itertools.combinations(combination_list[0], 2))) > 1000:
        print(f'Lots of Combos, ')
    
    #return
    shortest_dist = np.inf
    candidate_0_dist = np.inf
    pair_dict = {}
    
    if skip_nonroot:
        skip_dict = dict(zip(candidate_ids, candidate_ids))
    
    for idx, combination in enumerate(combination_list):
        pairs = list(itertools.combinations(combination, 2))
        if skip_nonroot:
            pairs = [(i,j) for i,j in pairs if skip_dict.get(i, None) is not None or 
                     skip_dict.get(j, None) is not None]
            
        if verbose: print(f'processing {len(pairs)} pairs at idx {idx}\t{time.time()-st}\tcurrent shortest: {shortest_dist}')
        
        distance = []
        for pair in pairs:
            if pair in pair_dict:
                pass
            else:
                pair_dict[pair] = get_cos(pair[0], pair[1])
            
            distance.append(pair_dict[pair])
            
        distance = sum(distance)/len(distance)
            
        if distance < shortest_dist:
            shortest_dist = distance
            candidate_id = combination[0]
            if idx == 0: candidate_0_dist = shortest_dist
            if verbose: print(f'candidate_id: {candidate_id}\tnew shortest: {shortest_dist}')

            if shortest_dist == 0:
                if verbose: print(f'Early Break {idx}: {candidate_id} {true_id}')
                break
    
    if (candidate_0_dist-shortest_dist) < tune*candidate_0_dist:
        candidate_id = candidate_ids[0]
    
    if tune == 0:
        return candidate_id == true_id
    elif candidate_id != candidate_ids[0]:
        if candidate_id == true_id:
            return 'shortest correct', shortest_dist, candidate_0_dist
        elif candidate_ids[0] == true_id:
            return 'cand_0 correct', shortest_dist, candidate_0_dist

a = [disambiguate(i, max_neighbors=4, max_neighbor_ids=8, max_root_ids=8) for i in get_samples(dataset, 250, seed=2)]
sum(a)/len(a)

0.6

In [36]:
def run_model(samples_per_iter, iterations, seed,
              max_neighbors, max_neighbor_ids, max_root_ids, skip_nonroot, \
              tune=0, compare=False, verbose=False):
    
    disambiguate_part = partial(disambiguate, max_neighbors=max_neighbors, max_neighbor_ids=max_neighbor_ids, \
                                max_root_ids=max_root_ids, skip_nonroot=skip_nonroot, tune=tune)
    scores = []
    for i in range(iterations):
        st = time.time()
        
        if seed is not None:
            samples = get_samples(dataset, samples_per_iter, seed=seed+i)
        else:
            samples = get_samples(dataset, samples_per_iter)

        p = mp.Pool(n_cores-1)
        results = p.map(disambiguate_part, samples)
        p.close()
        p.join()
        
        if tune == 0:
            model_score = sum(results)/len(results)

            if compare:
                baseline = [disambiguate(i, max_neighbors, max_neighbor_ids, max_root_ids, baseline=True) for i in samples]
                base_score = sum(baseline)/len(baseline)
                if verbose: print(f'model results: {model_score}\tbaseline results: {base_score}\ttime:{time.time()-st}')
                scores.append([model_score, base_score])
            else:
                if verbose: print(f'model results: {model_score}\ttime:{time.time()-st}')
                scores.append(model_score)   
        else:
            print(f'Run: {i}\tsamples: {samples_per_iter}\tTime:{time.time()-st}')
            tune_result = [j for j in results if type(j)==tuple]
            tune_df = pd.DataFrame(tune_result, columns=['result', 'short_dist', 'cand_0_dist'])
            tune_df['diff'] = tune_df.cand_0_dist-tune_df.short_dist
            tune_df['diff_pct'] = tune_df['diff']/tune_df['cand_0_dist']
            for col in tune_df.columns:
                print(f'{col}')
                display(tune_df.groupby('result')[col].describe())
            
    return scores

In [28]:
import multiprocessing as mp
from functools import partial 
n_cores = mp.cpu_count()
n_cores

16

In [67]:
%%time
# Tune based on when embedding and candidate 0 don't match

tune_result = run_model(1000, 1, 515, \
                       max_neighbors=8, max_neighbor_ids=10, max_root_ids=10, skip_nonroot=False, tune=0.4)

"""tune_result = [disambiguate(i, max_neighbors=4, max_neighbor_ids=10, max_root_ids=10, \
                            skip_nonroot=False, tune=-1) \
               for i in get_samples(dataset, 1000, seed=None)]

tune_df = pd.DataFrame([i for i in tune_result if type(i)==tuple], columns=['result', 'short_dist', 'cand_0_dist'])
tune_df['diff'] = tune_df.cand_0_dist-tune_df.short_dist
tune_df['diff_pct'] = tune_df['diff']/tune_df['cand_0_dist']
for i in tune_df.columns:
    display(i, tune_df.groupby('result')[i].describe())"""

Process ForkPoolWorker-263:
Process ForkPoolWorker-264:
Process ForkPoolWorker-258:
Process ForkPoolWorker-262:
Process ForkPoolWorker-257:
Process ForkPoolWorker-259:
Process ForkPoolWorker-269:
Process ForkPoolWorker-271:
Process ForkPoolWorker-270:
Process ForkPoolWorker-272:
Process ForkPoolWorker-273:
Process ForkPoolWorker-261:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


In [54]:
%%time
run_model(samples_per_iter = 500, iterations=5, seed=None, max_neighbors=4, max_neighbor_ids=32, max_root_ids=8, skip_nonroot=False, compare=True, verbose=True)

Process ForkPoolWorker-382:
Process ForkPoolWorker-377:
Process ForkPoolWorker-384:
Process ForkPoolWorker-376:
Process ForkPoolWorker-390:
Process ForkPoolWorker-381:
Process ForkPoolWorker-379:
Process ForkPoolWorker-389:
Process ForkPoolWorker-378:
Process ForkPoolWorker-380:
Process ForkPoolWorker-383:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Tracebac

Process ForkPoolWorker-388:
Process ForkPoolWorker-387:
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
Process ForkPoolWorker-385:
KeyboardInterrupt
Traceback (most recent call last):
KeyboardInterrupt
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multi

KeyboardInterrupt: 

  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ubuntu/miniconda3/envs/dev/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
KeyboardInterrupt
  File "/home/ubuntu/miniconda3/envs/de